# Recurrent Neural Networks and LSTMs/GRUs

Following [Classifying Names with a Character-Level RNN](https://docs.pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html)

For each character, generate a prediction and hidden state at each step by feeding its *previous* hidden state in each next step. Take the final prediction to be the output, which class the word belongs to.

The task is to train on thousands of last names from 18 different languages and predict which language a name is based on the spelling.

In [1]:
import torch

# Check if CUDA is available
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')

torch.set_default_device(device)
print(f"Using device = {torch.get_default_device()}")

Using device = cpu


# Data preparation